# image alignment tests


In [1]:
import cv2
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import napari

from align import *

In [2]:
folder='E:/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/SEM Image/'
folder='E:/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/Images/EDS/'
folder='E:/2020_11_18 C3S 28d+EDX (2 Teile)/2020_11_19 C3S 28d+EDX/EDS Export/'
#folder='E:/SEM Image/'
folder='F:/2020_11_19 C3S 28d+EDX/EDS Export/'
sem_folder='F:/2020_11_19 C3S 28d+EDX/Images/SEM Image/'

In [3]:
folder, eds_elements = check_folder_structure(folder)

identified 14 elements


In [4]:
sem_translation, error_list, filled_canvas, loaded_images, scaling = process_translation_of_folder(folder=folder, multicore=True, do_nlm=False, mask_size=0.99, eq_hist=True, crop_thresh=0 )

im_cnt = len(loaded_images)
z_scale=192

scale=(z_scale, scaling['y'], scaling['x'])

loading 80 images...
((96000, 1000),)
((96000, 1000),)
  Image is an EDS image of the FIB process
  26.943 x 26.943 nm/px

Found existing translation csv, loading...
Create 3D image stack with corrected image translation
  - allocating 3D image space..
  - translating and denoising images..
saving images..
saved "F:/2020_11_19 C3S 28d+EDX/EDS Export/Images\aligned\aligned_stack_(80).tif"
sucessfull


# add a shape and mark the top line of the cube

### correction of the y-translation

In [6]:
translation   = sem_translation.copy()

translation_csv = folder + os.sep + 'translations_fixed_y.csv'
fixed_translation = load_translation_csv( translation_csv, im_cnt )
if len(fixed_translation) != im_cnt-1:
    with napari.gui_qt():
        viewer = napari.view_image(filled_canvas, name='select y correction line', scale=scale )

    correction_dict = {}
    for point in viewer.layers[1].data[0]:
        correction_dict[math.floor(point[0])] = point[1]

    y_correction_list = get_axis_correction_list( correction_dict, im_cnt )

    for i, y in enumerate(y_correction_list):
        # print(i, y, translation[i-1][1], translation[i-1][1] - y)
        if i>0: translation[i-1][1] -= y
    save_translation_csv( translation, translation_csv)
else:
    translation = fixed_translation

Found existing translation csv, loading...


### correction of the x-translation

In [7]:
translation_csv = folder + os.sep + 'translations_fixed_xy.csv'
fixed_translation = load_translation_csv( translation_csv, im_cnt )
if len(fixed_translation) != im_cnt-1:
    corrected_images = create_3D_stack(translation, loaded_images, False)
    with napari.gui_qt():
        viewer = napari.view_image(corrected_images, name='select x correction line', scale=scale )
        viewer.scale_bar.visible = True

    correction_dict = {}
    for point in viewer.layers[1].data[0]:
        correction_dict[math.floor(point[0])] = point[0]

    x_correction_list = get_axis_correction_list( correction_dict, z_slice_count )

    for i, x in enumerate(x_correction_list):
        # print(i, y, translation[i-1][2], translation[i-1][2] - x)
        if i>0: translation[i-1][2] += x
    save_translation_csv( translation, translation_csv)

    # show final stack arrangement
    corrected_sem_images = create_3D_stack(translation, loaded_images, False)
    with napari.gui_qt():
        viewer = napari.view_image(corrected_sem_images, name='test2', scale=scale)
        viewer.scale_bar.visible = True
else:
    translation = fixed_translation
    corrected_sem_images = create_3D_stack(translation, loaded_images, False)

Found existing translation csv, loading...
Create 3D image stack with corrected image translation
  - allocating 3D image space..
  - translating and denoising images..


In [8]:
# sadly the automatic translation did not work
"""
print(round(len(loaded_images)/3), round(len(loaded_eds_images)/3))

#1 get representative SEM image
img_sem = loaded_images[19]

#2 get same plane for a specific element EDS images
selected_element = "Pt"
folder = eds_elements[selected_element]
_, loaded_eds_images = load_image_set(folder)
img_pt  = loaded_eds_images[1]

clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(1,1))
#3 get thresholded images of both images
img_sem = cv2.GaussianBlur(clahe.apply(cv2.medianBlur(img_sem, 5)), (5, 5), cv2.BORDER_DEFAULT)
img_pt  = cv2.GaussianBlur(clahe.apply(cv2.medianBlur(img_pt,  9)),  (15, 15), cv2.BORDER_DEFAULT)

#_, img_sem_bin = cv2.threshold( clahe.apply(cv2.medianBlur(img_sem, 5)) , 125, 255, cv2.THRESH_BINARY)
#_, img_pt_bin  = cv2.threshold( clahe.apply(cv2.medianBlur(img_pt,  5)) ,  60, 255, cv2.THRESH_BINARY)

col = 2
row = 1
fig=plt.figure(figsize=(col*5, row*5))
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
            
plt.margins(0,0)
plt.axis('off')
fig.add_subplot(row, col, 1)

plt.axis('off')
plt.imshow(img_sem, cmap='gray')

fig.add_subplot(row, col, 2)
plt.axis('off')
plt.imshow(img_pt, cmap='gray')

mask      = None
mask_full = None

#4 get the translation of the repective images
get_image_translation('asdf', img_sem, img_pt, mask, mask_full)
"""

'\nprint(round(len(loaded_images)/3), round(len(loaded_eds_images)/3))\n\n#1 get representative SEM image\nimg_sem = loaded_images[19]\n\n#2 get same plane for a specific element EDS images\nselected_element = "Pt"\nfolder = eds_elements[selected_element]\n_, loaded_eds_images = load_image_set(folder)\nimg_pt  = loaded_eds_images[1]\n\nclahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(1,1))\n#3 get thresholded images of both images\nimg_sem = cv2.GaussianBlur(clahe.apply(cv2.medianBlur(img_sem, 5)), (5, 5), cv2.BORDER_DEFAULT)\nimg_pt  = cv2.GaussianBlur(clahe.apply(cv2.medianBlur(img_pt,  9)),  (15, 15), cv2.BORDER_DEFAULT)\n\n#_, img_sem_bin = cv2.threshold( clahe.apply(cv2.medianBlur(img_sem, 5)) , 125, 255, cv2.THRESH_BINARY)\n#_, img_pt_bin  = cv2.threshold( clahe.apply(cv2.medianBlur(img_pt,  5)) ,  60, 255, cv2.THRESH_BINARY)\n\ncol = 2\nrow = 1\nfig=plt.figure(figsize=(col*5, row*5))\nplt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, \n            hspace = 0, wsp

In [9]:
"""
def get_full_img_translation( translation, search_pos, verbose = False ):
    x_t = 0
    y_t = 0
    min_x = 0
    min_y = 0
    if verbose: print('~'*20)
    for pos, step in enumerate(translation):
        if verbose: print(pos, (step[2], step[1]), (x_t+step[2], y_t+step[1]))
        if pos <= search_pos:
            x_t += step[2]
            y_t += step[1]
        if pos==search_pos:
            if verbose: print('this translation', step)
        if min_x > step[2]: min_x = step[2]
        if min_y > step[1]: min_y = step[1]

    return (min_x, x_t), (min_y, y_t)
print( get_full_img_translation( translation, 17, True ) )
"""

"\ndef get_full_img_translation( translation, search_pos, verbose = False ):\n    x_t = 0\n    y_t = 0\n    min_x = 0\n    min_y = 0\n    if verbose: print('~'*20)\n    for pos, step in enumerate(translation):\n        if verbose: print(pos, (step[2], step[1]), (x_t+step[2], y_t+step[1]))\n        if pos <= search_pos:\n            x_t += step[2]\n            y_t += step[1]\n        if pos==search_pos:\n            if verbose: print('this translation', step)\n        if min_x > step[2]: min_x = step[2]\n        if min_y > step[1]: min_y = step[1]\n\n    return (min_x, x_t), (min_y, y_t)\nprint( get_full_img_translation( translation, 17, True ) )\n"

## load EDS images

In [10]:
eds_x_offset = 404.8#+22
eds_y_offset = 50.8
#expected_img_count = 63
relevant_elements = ['Ga', 'Si', 'O', 'Pt', 'C']
colormaps = ["red", "green", "blue", "yellow", "magenta"]

if len(eds_elements) > 0:

    scale=(z_scale, scaling['y'], scaling['x'])
    image_stacks = {}
    for element in eds_elements.keys():
        if element in relevant_elements:
            stack = process_element(eds_elements, element, translation, eds_x_offset, eds_y_offset)
            if len(stack) != len(loaded_images):
                print('Dataset for {} is incomplete ({} != {})'.format(element, len(stack), len(loaded_images)))
            else:
                image_stacks[element] = stack
        else:
            print('ignoring {}'.format(element))

    # show image stacks
    print('translate and display image stack using the following voxel dimensions: x={:.2f}, y={:.2f}, z={:.2f} {}'.format(scale[2], scale[1], scale[0], scaling['unit']))
    #scale=(z_scale*eds_each_nth_slice, x_scale/math.cos(38), x_scale)
    with napari.gui_qt():
        viewer = napari.view_image(corrected_sem_images, name='SEM images', scale=scale)
        for i, selected_element in enumerate(image_stacks):
            print(i, selected_element)
            new_layer = viewer.add_image(image_stacks[selected_element], name='EDS results for {}'.format(selected_element), scale=scale, colormap=colormaps[i], opacity=1, blending="additive", rendering="iso")
        viewer.scale_bar.visible = True
else:
    print('no elements found')

ignoring Ag
ignoring Al
ignoring Ar
--------------------
selected element is C
loading 80 images...
 Found multiple shapes!
  found 17 images with this shape: (654, 565)
  found 63 images with this shape: (624, 515)
  - eds x translation: 404.8 - -132.3115293083519 + -65.33373174191244 
  - eds y translation: 50.8 - -24.07718880962004 + -22.62759310139859 
  - final translation 471.77779756643946 and 52.24959570822145
1.4495957082214517
['Elektronenbild 21.tif', 1.4495957082214526, 2.010848378460873]
 denoising images and enhance contrast
Create 3D image stack with corrected image translation
found first_x_offset: 471.77779756643946
found first_y_offset: 52.24959570822145
  - allocating 3D image space..
  - translating and denoising images..
ignoring Ca
ignoring Cl
ignoring Cu
--------------------
selected element is Ga
loading 80 images...
 Found multiple shapes!
  found 17 images with this shape: (654, 565)
  found 63 images with this shape: (624, 515)
  - eds x translation: 404.8 - 

In [25]:
folder = sem_folder
translation, error_list, filled_canvas, loaded_images, scaling = process_translation_of_folder(folder=folder, multicore=True, do_nlm=False, mask_size=0.99, eq_hist=True, crop_thresh=0 )
print(len(loaded_images))

im_cnt = len(loaded_images)
z_scale=19.2
scale=(z_scale, scaling['y'], scaling['x'])
print(scale)

loading 672 images...

trying to detect FEI scaling
SEM image saved by an FEI / thermoScientific device - probaply created by FIB process
Found existing translation csv, loading...
Create 3D image stack with corrected image translation
  - allocating 3D image space..
  - translating and denoising images..
saving images..
saved "F:/2020_11_19 C3S 28d+EDX/Images/SEM Image/aligned\aligned_stack_(672).tif"
sucessfull
672
(19.2, 9.41609063976502, 8.993060000000002)


In [26]:
redo_translation = False

In [27]:
translation_csv = folder + os.sep + 'translations_fixed_y.csv'
fixed_translation = load_translation_csv( translation_csv, im_cnt )
if (len(fixed_translation) != im_cnt-1 or redo_translation):
    normed_scale = scale#(1, 1, 1)
    with napari.gui_qt():
        viewer = napari.view_image(filled_canvas, name='select y correction line', scale=normed_scale )

    correction_dict = {}
    for point in viewer.layers[1].data[0]:
        correction_dict[math.floor(point[0])] = point[1]

    y_correction_list = get_axis_correction_list( correction_dict, im_cnt )

    for i, y in enumerate(y_correction_list):
        #print(i, y, translation[i-1][1], translation[i-1][1] - y)
        if i>0: translation[i-1][1] -= y
    save_translation_csv( translation, translation_csv)
else:
    translation = fixed_translation

-0.31394031726714866 0.31342772265619123
355 -0.6273680399233399 -1.0106449053707016 -0.3832768654473617
356 -0.6273680399233399 0.36430817285289613 0.991676212776236
357 -0.6273680399233399 0.22529582089074074 0.8526638608140806
358 -0.6273680399233399 1.2466155388520024 1.8739835787753423
359 -0.6273680399233399 0.4708566173483356 1.0982246572716754
360 -0.6273680399233399 0.18838570717470593 0.8157537470980458
361 -0.6273680399233399 -7.697969199314815 -7.070601159391475
362 -0.6273680399233399 -0.8307530728763849 -0.203385032953045
363 -0.6273680399233399 -1.06941703063821 -0.44204899071487014
364 -0.6273680399233399 2.559778766351029 3.187146806274369
365 -0.6273680399233399 0.2125217637514698 0.8398898036748097
366 -0.627368039923283 0.7031734357202294 1.3305414756435123
367 -0.6273680399233399 1.538922041300174 2.166290081223514
368 -0.6273680399233399 -0.7712654847502413 -0.14389744482690137
369 -0.6273680399233399 0.27382313065378294 0.9011911705771228
370 -0.6273680399233399 

In [28]:
translation_csv = folder + os.sep + 'translations_fixed_xy.csv'
fixed_translation = load_translation_csv( translation_csv, im_cnt )
if (len(fixed_translation) != im_cnt-1 or redo_translation):
    normed_scale = scale#(1, 1, 1)
    corrected_images = create_3D_stack(translation, loaded_images, False)
    with napari.gui_qt():
        viewer = napari.view_image(corrected_images, name='select x correction line', scale=normed_scale )
        viewer.scale_bar.visible = True

    correction_dict = {}
    for point in viewer.layers[1].data[0]:
        correction_dict[math.floor(point[0])] = point[0]

    x_correction_list = get_axis_correction_list( correction_dict, z_slice_count )

    for i, x in enumerate(x_correction_list):
        # print(i, x, translation[i-1][2], translation[i-1][2] - x)
        if i>0: translation[i-1][2] += x
    save_translation_csv( translation, translation_csv)

    
else:
    translation = fixed_translation


Found existing translation csv, loading...
Create 3D image stack with corrected image translation
  - allocating 3D image space..
  - translating and denoising images..
saving translation matrix to F:/2020_11_19 C3S 28d+EDX/Images/SEM Image/\translations_fixed_xy.csv
saved F:/2020_11_19 C3S 28d+EDX/Images/SEM Image/\translations_fixed_xy.csv
Create 3D image stack with corrected image translation
  - allocating 3D image space..
  - translating and denoising images..


In [30]:


# preprocess the raw eds images - denoising, contrast enhancing and segmentation
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(1,1))
def preprocess_sem_image(img):
    tmp = clahe.apply(cv2.medianBlur(img, 9))

    return clahe.apply( tmp )
    
for i, img in enumerate(loaded_images):
    loaded_images[i] = preprocess_sem_image(img)


# final stack arrangement
corrected_sem_images = create_3D_stack(translation, loaded_images, False)

Create 3D image stack with corrected image translation
  - allocating 3D image space..
  - translating and denoising images..


In [31]:
print(scale)
with napari.gui_qt():
    viewer = napari.view_image(corrected_sem_images, name='high resolution stack', scale=scale)
    viewer.scale_bar.visible = True

(19.2, 9.41609063976502, 8.993060000000002)
C:\Users\ELM\AppData\Roaming\Python\Python38\site-packages\napari\_vispy\vispy_image_layer.py:204: UserWarning: data shape (672, 2314, 2118) exceeds GL_MAX_TEXTURE_SIZE 2048 in at least one axis and will be downsampled. Rendering is currently in 3D mode.
  warnings.warn(


refFilename = "SEM Image - SliceImage - 001.tif"
print("Reading reference image : ", refFilename)
im_a = cv2.imread(refFilename, cv2.IMREAD_GRAYSCALE)
print(im_a.shape)

# Read image to be aligned
imFilename = "SEM Image - SliceImage - 003.tif"
print("Reading image to align : ", imFilename);  
im_b = cv2.imread(imFilename, cv2.IMREAD_GRAYSCALE)
print(im_b.shape)


#im_a_denoised = im_a#denoiseNLMCV2( im_a )
#im_b_denoised = im_b#denoiseNLMCV2( im_b )
gauss_kernel = (3, 3)
im_a_denoised = cv2.GaussianBlur(im_a, gauss_kernel, cv2.BORDER_DEFAULT)
im_b_denoised = cv2.GaussianBlur(im_b, gauss_kernel, cv2.BORDER_DEFAULT)

mask = get_centered_mask(im_a, mask_size = 0.75)
masked_im_a = cv2.bitwise_and(im_a_denoised, im_a_denoised, mask = mask)
masked_im_b = cv2.bitwise_and(im_b_denoised, im_b_denoised, mask = mask)


#fig=plt.figure(figsize=(col*5, row*5))
#plt.imshow(im_b, cmap='gray')

col = 2
row = 3
fig=plt.figure(figsize=(col*5, row*5))
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
            
plt.margins(0,0)
plt.axis('off')
fig.add_subplot(row, col, 1)
plt.axis('off')
plt.imshow(im_a, cmap='gray')
fig.add_subplot(row, col, 2)
plt.axis('off')
plt.imshow(im_b, cmap='gray')
fig.add_subplot(row, col, 3)
plt.axis('off')
plt.imshow(im_a_denoised, cmap='gray')
fig.add_subplot(row, col, 4)
plt.axis('off')
plt.imshow(im_b_denoised, cmap='gray')
fig.add_subplot(row, col, 5)
plt.axis('off')
plt.imshow(masked_im_a, cmap='gray')
fig.add_subplot(row, col, 6)
plt.axis('off')
plt.imshow(masked_im_b, cmap='gray')

# Registered image will be resotred in imReg. 
# The estimated homography will be stored in h. 
print(im_b_denoised.shape)
imReg, h = alignImages(im_b_denoised, im_a_denoised, mask=mask)
print(imReg.shape)


fig=plt.figure(figsize=(10, 10))
plt.imshow(imReg, cmap='gray')


t_x = h[0,2]
t_y = h[1,2]
# only extract x and y movement
# https://stackoverflow.com/questions/25658443/calculating-scale-rotation-and-translation-from-homography-matrix
h_new=np.array([[1,0,t_x],[0,1,t_y],[0,0,1]])
print(math.ceil(t_x),math.ceil(t_y))
# Use homography
height, width = im_b_denoised.shape
im1Reg = cv2.warpPerspective(im_b_denoised, h_new, (math.ceil(width+t_x), math.ceil(height+t_y)))
print(im1Reg.shape)

fig=plt.figure(figsize=(10, 10))
plt.imshow(im1Reg, cmap='gray')

immatches = cv2.imread('matches.tif', cv2.IMREAD_COLOR)
fig=plt.figure(figsize=(10 , 5))
plt.imshow(immatches)

# Write aligned image to disk. 
outFilename = "aligned.tif"
print("Saving aligned image : ", outFilename); 
cv2.imwrite(outFilename, imReg)